In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
#from keras.layers import GRU
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/disney/music_train/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open("drive/My Drive/Desktop/projet-ANNUEL/Create_song/data/notes", 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    #
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    #
    model.add(Dense(256))    #
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/disney/weight/weights-FF-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=300, batch_size=500, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()


Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/disney/music_train/Aladdin-AradianNights.mid
Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/disney/music_train/CircleOfLife-1.mid
Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/disney/music_train/25958_Friend-Like-Me.mid
Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/disney/music_train/drl_hakuna_matata.mid
Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/disney/music_train/ThisLand-1.mid


W0626 21:03:20.545305 139662634219392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0626 21:03:20.560929 139662634219392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0626 21:03:20.563612 139662634219392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0626 21:03:21.349506 139662634219392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0626 21:03:21.359029 

Epoch 1/300
3391/3391 [==============================] - 10s 3ms/step - loss: 7.0307
Epoch 2/300
3391/3391 [==============================] - 6s 2ms/step - loss: 4.2249
Epoch 3/300
3391/3391 [==============================] - 6s 2ms/step - loss: 4.2329
Epoch 4/300
3391/3391 [==============================] - 6s 2ms/step - loss: 4.2054
Epoch 5/300
3391/3391 [==============================] - 6s 2ms/step - loss: 4.1803
Epoch 6/300
3391/3391 [==============================] - 6s 2ms/step - loss: 4.2147
Epoch 7/300
3391/3391 [==============================] - 6s 2ms/step - loss: 4.1568
Epoch 8/300
3391/3391 [==============================] - 6s 2ms/step - loss: 4.2143
Epoch 9/300
 500/3391 [===>..........................] - ETA: 5s - loss: 4.1079

KeyboardInterrupt: ignored

In [0]:
notes = get_notes()

    # get amount of pitch names
n_vocab = len(set(notes))
print(n_vocab)

Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/bach/bach_bourree.mid
Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/bach/bach_tocatta_fugue_d_minor.mid
Parsing drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/bach/bach solfeggietto.mid
113


In [0]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
#from keras.layers import GRU
from keras.layers import Activation


def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('drive/My Drive/Desktop/projet-ANNUEL/Create_song/data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """
    Prepare the sequences used by the Neural Network
    :param notes: dgdfgdfgdfgdfgdfg
    :return: dfgdfgdfgdfgdfgdfgdfgdf
    """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    #
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    #
    model.add(Dense(256))    #
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/the lord of the rings/pt1/weight/weights-FF-296-0.0552-bigger.hdf5')

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/the lord of the rings/pt1/midi output/test_outputstp296.mid')

if __name__ == '__main__':
    generate()

In [0]:
import os
os.getcwd()

'/content'

In [0]:
!pip install midi2audio

In [0]:
!pip install fluidsynth
from midi2audio import FluidSynth

In [0]:
FluidSynth().play_midi('drive/My Drive/Desktop/projet-ANNUEL/Create_song/midi_songs/ff8/output_music/ff8.mid')

FileNotFoundError: ignored